# Iterative process - Text classification from manual rules to SoA in 2022

<sup>Introduction to Data Science</sub>

Text classification is normally referred to as deriving a class for a text document according to precalculated features. A text document can be a paragraph, tweet post, sentence, forum post, ... On the other hand, text tagging problems deal with classification of tokens in a sequence. Sequences can represent tokens, sentences, forum thread, characters, ...

## Text Classification

Some examples of text classification problems are text categorization, spam detection, sentiment analysis, language identification, ... Definition of a problem:

**Input:**
* a document $d \in D$
* a fixed set of classes $C=\{c_1, c_2, c_3, ... c_j\}$

**Output:**
* predicted class $c \in C$

<center>
    <img src="bow1.png" width="600">
</center>

## Data description

> IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms.
For more dataset information, please go through the following link, [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).



In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import stanza
import nltk
import string
from nltk.corpus import stopwords
from nltk import FreqDist
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from transformers import (TFBertForSequenceClassification, BertTokenizer)
from tqdm import tqdm
from keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf

nltk.download("punkt")
nltk.download("stopwords")
stanza.download("en")

[nltk_data] Downloading package punkt to /home/slavkoz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/slavkoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2022-04-13 22:54:36 INFO: Downloading default packages for language: en (English)...
2022-04-13 22:54:37 INFO: File exists: /home/slavkoz/stanza_resources/en/default.zip.
2022-04-13 22:54:41 INFO: Finished downloading models and saved to /home/slavkoz/stanza_resources.


In [2]:
data = pd.read_csv('IMDB Dataset.csv')

# Transform positive/negative values to 1/0s
data['sentiment'] = LabelEncoder().fit_transform(data['sentiment'])

print(f"Corpus: \n{data}\n\n\n")
print(f"Example 1:\n\tText:\n\t\t{data['review'][666]}\n\tClass:\n\t\t{data['sentiment'][666]}")

Corpus: 
                                                  review  sentiment
0      One of the other reviewers has mentioned that ...          1
1      A wonderful little production. <br /><br />The...          1
2      I thought this was a wonderful way to spend ti...          1
3      Basically there's a family where a little boy ...          0
4      Petter Mattei's "Love in the Time of Money" is...          1
...                                                  ...        ...
49995  I thought this movie did a down right good job...          1
49996  Bad plot, bad dialogue, bad acting, idiotic di...          0
49997  I am a Catholic taught in parochial elementary...          0
49998  I'm going to have to disagree with the previou...          0
49999  No one expects the Star Trek movies to be high...          0

[50000 rows x 2 columns]



Example 1:
	Text:
		This was a fine example of how an interesting film can be made without using big stars and big effects. Just tell a true story

Prepare train and test data.

In [3]:
X = (np.array(data['review']))
y = (np.array(data['sentiment']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"First training example: \n\t({X_train[0]}, {y_train[0]})")

First training example: 
	(As much as I love trains, I couldn't stomach this movie. The premise that one could steal a locomotive and "drive" from Arkansas to Chicago without hitting another train along the way has to be right up there on the Impossible Plot lines hit board. Imagine two disgruntled NASA employees stealing the "crawler" that totes the shuttles to and fro and driving it to New York and you get the idea.<br /><br />Having said all that, it's a nice try. Wilford Brimely is at his Quaker Oats best, and Levon Helm turns a good performance as his dimwitted but well-meaning sidekick. Bob Balaban is suitably wormy as the Corporate Guy, and the "little guy takes on Goliath" story gets another airing., 0)


In [4]:
print(f"Train dataset shape: {X_train.shape}, \nTest dataset shape: {X_test.shape}")

Train dataset shape: (35000,), 
Test dataset shape: (15000,)


### (1) Rule-based approach

<center>
    <img src="light_bulb.jpg" width="300" />
</center>

Let's say that we can identify positive and negative reviews based on positive and negative keywords.

In [5]:
def get_word_list(filename):
    return [word.strip() for word in open(filename,'r').read().splitlines() if word.strip() != "" and not word.startswith(";")] 

positive_words = get_word_list("positive-words.txt")
negative_words = get_word_list("negative-words.txt")

print(f"First 10 positive words: {', '.join(positive_words[0:10])}.")
print(f"First 10 negative words: {', '.join(negative_words[0:10])}.")

First 10 positive words: a+, abound, abounds, abundance, abundant, accessable, accessible, acclaim, acclaimed, acclamation.
First 10 negative words: 2-faced, 2-faces, abnormal, abolish, abominable, abominably, abominate, abomination, abort, aborted.


Create classifier

In [6]:
def classify(review):
    pos_counts = sum([review.count(word) for word in positive_words])
    neg_counts = sum([review.count(word) for word in negative_words])
    
    if pos_counts > neg_counts:
        return 1
    else:
        return 0

Predict on test data

In [ ]:
predictions_rb = [classify(review) for review in X_test]

In [8]:
print(f"First 10 predictions: {' '.join(map(str, predictions_rb[0:10]))}.")
print(f"First 10 test labels: {' '.join(map(str, y_test[0:10]))}.")

First 10 predictions: 0 0 0 1 0 0 0 0 0 0.
First 10 test labels: 1 1 0 1 0 1 1 1 0 0.


#### Performance measures

Most common general measures for the classification methods in the machine learning are classification accuracy, precision, recall and F-score. In general you select a measure 

Let's imagine we have a problem with two target classes. We would calculate performance of our classifier as follows:

| - |Correct  |Not Correct  |
|-------------|:-------:|:-----------:|
|Selected     |TP       | FP          |
|Not selected |FN       | TN          |

Classification accuracy:
$$
\textrm{CA} = \frac{\textrm{TP}+\textrm{TN}}{\textrm{TP}+\textrm{FP}+\textrm{FN}+\textrm{TN}}
$$

Precision or Sensitivity: a proportion of selected items that are correct (in medical terms probability of being test positive when disease present)
$$
\textrm{P} = \textrm{Sensitivity} = \frac{\textrm{TP}}{\textrm{TP}+\textrm{FP}}
$$

Recall: a proportion of correct items that are selected
$$
\textrm{R} = \frac{\textrm{TP}}{\textrm{TP}+\textrm{FN}}
$$

Specificity: a proportion of not correct items that are not selected (in medical terms probability of being test negative when disease absent)
$$
\textrm{Specificity} = \frac{\textrm{TN}}{\textrm{FP}+\textrm{TN}}
$$

F (i.e. $F_1$) measure: a harmonic mean (i.e., very conservative average) between precision and recall
$$
\textrm{F} = \frac{(\beta^2+1)\textrm{PR}}{\beta^2\textrm{P}+\textrm{R}}; \textrm{F}_1 = \frac{2\textrm{PR}}{\textrm{P}+\textrm{R}}
$$

In [9]:
def print_performance_report(y_true, y_pred):
    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    p = metrics.precision_score(y_true, y_pred)
    r = metrics.recall_score(y_true, y_pred)
    print(f"\tAccuracy: {acc:.2}, Precision: {p:.2}, Recall: {r:.2}, F1 score: {f1:.2}")

In [10]:
print("Rule-based approach")
print_performance_report(y_test, predictions_rb)

Rule-based approach
	Accuracy: 0.66, Precision: 0.83, Recall: 0.41, F1 score: 0.55


Always define a baseline to compare to

In [11]:
print("Random classifier")
print_performance_report(y_test, [random.randint(0, 1) for i in range(len(y_test))])

Random classifier
	Accuracy: 0.5, Precision: 0.5, Recall: 0.5, F1 score: 0.5


### (2) Improved rule-based approach

In positive and negative lists we have lemmas that might not be present in review texts. What is the performance if we lemmatize reviews?

In [12]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

2022-04-13 22:58:35 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-04-13 22:58:35 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2022-04-13 22:58:35 INFO: Use device: gpu
2022-04-13 22:58:35 INFO: Loading: tokenize
2022-04-13 22:58:37 INFO: Loading: pos
2022-04-13 22:58:37 INFO: Loading: lemma
2022-04-13 22:58:37 INFO: Done loading processors!


In [15]:
print(f"First movie review: \n\t'{X_test[0]}'.'", end="\n\n")
print(f"First movie review (lemmatized): \n\t'{' '.join([word.lemma for sent in nlp(X_test[0]).sentences for word in sent.words])}'.'")

First movie review: 
	'I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, an

In [20]:
predictions_irb = []

for id, review in enumerate(X_test):
    lemmas = []
    for sent in nlp(review).sentences:
        for word in sent.words:
            lemmas.append(word.lemma)
            
    predictions_irb.append(classify(lemmas))
    
    if id % 100 == 0:
        print(".", end="")     

......................................................................................................................................................

In [21]:
print(f"First 10 predictions: {' '.join(map(str, predictions_irb[0:10]))}.")
print(f"First 10 test labels: {' '.join(map(str, y_test[0:10]))}.")

First 10 predictions: 0 1 0 1 0 1 1 1 0 1.
First 10 test labels: 1 1 0 1 0 1 1 1 0 0.


In [22]:
print("Improved rule-based approach (lemmatization)")
print_performance_report(y_test, predictions_irb)

Improved rule-based approach (lemmatization)
	Accuracy: 0.73, Precision: 0.74, Recall: 0.72, F1 score: 0.73


### (3) Traditional approaches - naive Bayes, bag of words

One of the simplest classification methods based on a Bayes rule. It assumes features independence and therefore it can lack performance (e.g. XOR problem).

$$
P(c|d) = \frac{P(d|c)P(c)}{P(d)} \\
c_{MAP} = \textrm{argmax}_{c \in C}P(x_1|c)P(x_2|c)P(x_3) ... P(x_n|c)P(c)
$$

For the classification we need to define the features - i.e. representation of an input document for a classifier. A simple approach would be to use a bag of words representation:

<center>
 <img src="bow2.png" width="600">
</center>

Select words that will be taken into account

In [23]:
tokens = []

words_to_remove = stopwords.words('english') + list(string.punctuation)
chars_to_remove = set(string.punctuation)

for review in X_train:
    tokens += [token for token in nltk.word_tokenize(review.lower()) \
                   if token not in words_to_remove and \
                      len(set(token).intersection(chars_to_remove)) == 0]

In [24]:
freq = FreqDist(tokens)

vocabulary_size = 1000
vocabulary = [word for (word, occurences) in sorted(freq.items(), key = lambda x: x[1], reverse = True)[:vocabulary_size]]

print(f"Top 20 vocabulary words: {', '.join(vocabulary[:20])}.")

Top 20 vocabulary words: br, movie, film, one, like, good, would, even, time, really, see, story, well, much, could, get, people, bad, also, great.


Train classifier

In [25]:
# Prepare features
def prepare_feature_vector(texts, vocabulary):
    examples_features = []
    for example in texts:
        example_features = []
        for word in vocabulary:
            example_features.append(example.count(word))
        examples_features.append(example_features)
    return examples_features

X_train_bayes = prepare_feature_vector(X_train, vocabulary)
X_test_bayes = prepare_feature_vector(X_test, vocabulary)

In [26]:
print(f"Each document is represented as a vector of length: {len(X_train_bayes[0])}\n")
print(f"First document: \n\t{X_train[0]}\n")
print(f"First 20 vocabulary words: {', '.join(vocabulary[:20])}\n")
print(f"First 20 features: {', '.join(map(str, X_train_bayes[0][:20]))}")

Each document is represented as a vector of length: 1000

First document: 
	As much as I love trains, I couldn't stomach this movie. The premise that one could steal a locomotive and "drive" from Arkansas to Chicago without hitting another train along the way has to be right up there on the Impossible Plot lines hit board. Imagine two disgruntled NASA employees stealing the "crawler" that totes the shuttles to and fro and driving it to New York and you get the idea.<br /><br />Having said all that, it's a nice try. Wilford Brimely is at his Quaker Oats best, and Levon Helm turns a good performance as his dimwitted but well-meaning sidekick. Bob Balaban is suitably wormy as the Corporate Guy, and the "little guy takes on Goliath" story gets another airing.

First 20 vocabulary words: br, movie, film, one, like, good, would, even, time, really, see, story, well, much, could, get, people, bad, also, great

First 20 features: 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0


In [27]:
clf = MultinomialNB()

# Training
clf.fit(X_train_bayes, y_train)

MultinomialNB()

In [28]:
# Predicting
predictions_nb = clf.predict(X_test_bayes)

In [29]:
print("Traditional approach - naive Bayes (bag of words)")
print_performance_report(y_test, predictions_nb)

Traditional approach - naive Bayes (bag of words)
	Accuracy: 0.82, Precision: 0.82, Recall: 0.82, F1 score: 0.82


### (4) Traditional approaches - logistic regression, bag of words

Naive Bayes is one of the simplest algorithms that has some shortcomings. Alternatively we can also use other traditional algorithms, such as logistic regression, support-vector machines, k-nearest neighbours, classification tree, random forest, ...

In general logistic regression is a classifier to try.

In [30]:
scaler = preprocessing.StandardScaler().fit(X_train_bayes)
X_train_lg = scaler.transform(X_train_bayes)
X_test_lg = scaler.transform(X_test_bayes)

In [31]:
clf = LogisticRegression()
clf.fit(X_train_lg, y_train)
predictions_lg = clf.predict(X_test_lg)

In [32]:
print("Traditional approach - logistic regression (bag of words)")
print_performance_report(y_test, predictions_lg)

Traditional approach - logistic regression (bag of words)
	Accuracy: 0.86, Precision: 0.86, Recall: 0.87, F1 score: 0.86


### (5) Traditional approaches - logistic regression, TF-IDF

Instead of bag-of-words features we can introduce different features that weigh each word by importance for a document in a corpus. TF-IDF is a common metric in information retrieval which weights a term with respect to a list of documents.

> In information retrieval, tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in information retrieval and text mining. The tf-idf value increases proportionally to the number of times a word appears in the document, but is offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general.

> Variations of the tf–idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query. tf–idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.

The metric is calculated as follows:

$$\textrm{TF}(term) = \frac{\textrm{Number of times term appears in a document}}{\textrm{Total number of terms in the document}}$$

$$\textrm{IDF}(term) = \frac{\textrm{Total number of documents}}{\textrm{Number of documents with the term in it}}$$

$$\textrm{TF-IDF}(term) = \textrm{TF}(term) * \log_e(\textrm{IDF}(term))$$

In [33]:
# Extracting features from the data using a sparse vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
print("Training samples: %d, training features: %d" % X_train_tfidf.shape)
print("Testing samples: %d, Testing features: %d" % X_test_tfidf.shape)

Training samples: 35000, training features: 87684
Testing samples: 15000, Testing features: 87684


In [34]:
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
predictions_lg_tfidf = clf.predict(X_test_tfidf)

In [35]:
print("Traditional approach - logistic regression (TF-IDF)")
print_performance_report(y_test, predictions_lg_tfidf)

Traditional approach - logistic regression (TF-IDF)
	Accuracy: 0.9, Precision: 0.89, Recall: 0.91, F1 score: 0.9


### (6) Neural network-based approaches - LSTM

A neural network (NN) is built from nodes (neurons) stacked in layers between the feature vector and the target vector. The simplest version of NN consists of one node - *perceptron*.

The Perceptron is an algorithm for supervised learning of binary classifiers - functions that can decide whether an input (represented by a vector of numbers) belongs to one class or another. Much like logistic regression, the weights in a neural net are being multiplied by the input vertor summed up and feeded into the activation function's input.

A Perceptron Network can be designed to have multiple layers, leading to the Multi-Layer Perceptron (MLP).

<center>
    <img src="single_layer.png" width="60%" />
</center>

_(Source: Python Machine Learning, S. Raschka)_

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior.

<center>
    <img src="rnn.png" width="60%">
</center>

An LSTM network is an artificial neural network that contains LSTM blocks instead of, or in addition to, regular network units. A LSTM block may be described as a "smart" network unit that can remember a value for an arbitrary length of time. Unlike traditional RNNs, an Long short-term memory network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events.

<center>
    <img src="lstm.png" width="60%">
</center>

In [36]:
vocabulary_size = 20000
vocabulary = [word for (word, occurences) in sorted(freq.items(), key = lambda x: x[1], reverse = True)[:vocabulary_size]]

In [37]:
[word for (word, occurences) in sorted(freq.items(), key = lambda x: x[1], reverse = True)[:vocabulary_size]]

lstm_vocabulary = ["<PAD>", "<START>", "<UNK>"] + vocabulary
lstm_vocabulary_mapper = dict(zip(lstm_vocabulary, range(len(lstm_vocabulary))))
lstm_vocabulary_mapper_reverse = dict(zip(range(len(lstm_vocabulary)), lstm_vocabulary))

In [38]:
X_train_lstm = []
for review in X_train:
    example = [1] + \
        [lstm_vocabulary_mapper.get(token, 2) for token in nltk.word_tokenize(review.lower())]
    X_train_lstm.append(example)
    
X_test_lstm = []
for review in X_test:
    example = [1] + \
        [lstm_vocabulary_mapper.get(token, 2) for token in nltk.word_tokenize(review.lower())]
    X_test_lstm.append(example)

In [39]:
# number of words per review
max_review_length = 500

print(f'First review: \n\t{X_train_lstm[0]}.')
print(f'First label: {y_train[0]}.')
print(f'Length of first review before padding: {len(X_train_lstm[0])}.')

# padding reviews
X_train_lstm = sequence.pad_sequences(X_train_lstm, maxlen=max_review_length)
X_test_lstm = sequence.pad_sequences(X_test_lstm, maxlen=max_review_length)
print(f"\nAfter padding:")
print(f'First review: \n\t{X_train_lstm[0]}.')
print(f'Length of first review after padding: {len(X_train_lstm[0])}.')
print(f"Raw IDs representation: {[lstm_vocabulary_mapper_reverse.get(i) for i in X_train_lstm[0]]}\n\n")

First review: 
	[1, 2, 16, 2, 2, 39, 6324, 2, 2, 17, 2, 2905, 2, 4, 2, 2, 694, 2, 6, 17, 1893, 2, 2, 2, 2, 1376, 2, 2, 2, 2, 3237, 98, 3541, 64, 915, 216, 2, 26, 2, 2, 2, 100, 2, 2, 2, 2, 1015, 38, 289, 511, 1657, 2, 659, 35, 10843, 9231, 7659, 2823, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1585, 2, 2, 74, 702, 2, 2, 18, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 183, 2, 2, 2, 2, 2, 2, 209, 226, 2, 14679, 2, 2, 2, 2, 2, 2, 42, 2, 2, 2, 11481, 381, 2, 8, 131, 2, 2, 17643, 2, 2, 3716, 2, 1724, 2, 2, 5955, 2, 2, 2, 4717, 112, 2, 2, 2, 2, 40, 112, 190, 2, 2, 2, 14, 105, 64, 9882, 2].
First label: 0.
Length of first review before padding: 146.

After padding:
First review: 
	[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0  

In [40]:
model = Sequential()
model.add(Embedding(len(lstm_vocabulary), 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
print('Train...')
model.fit(X_train_lstm, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test_lstm, y_test))

Train...
Epoch 1/15
1094/1094 [==============================] - 1029s 939ms/step - loss: 0.5804 - accuracy: 0.6767 - val_loss: 0.6265 - val_accuracy: 0.6660
Epoch 2/15
1094/1094 [==============================] - 1028s 940ms/step - loss: 0.4481 - accuracy: 0.7992 - val_loss: 0.4910 - val_accuracy: 0.7599
Epoch 3/15
1094/1094 [==============================] - 1027s 939ms/step - loss: 0.3028 - accuracy: 0.8771 - val_loss: 0.3442 - val_accuracy: 0.8615
Epoch 4/15
1094/1094 [==============================] - 1027s 939ms/step - loss: 0.1878 - accuracy: 0.9330 - val_loss: 0.3212 - val_accuracy: 0.8779
Epoch 5/15
1094/1094 [==============================] - 1028s 939ms/step - loss: 0.1259 - accuracy: 0.9579 - val_loss: 0.3405 - val_accuracy: 0.8751
Epoch 6/15
1094/1094 [==============================] - 1027s 939ms/step - loss: 0.0870 - accuracy: 0.9729 - val_loss: 0.4161 - val_accuracy: 0.8707
Epoch 7/15
1094/1094 [==============================] - 1029s 941ms/step - loss: 0.0540 - accurac

In [42]:
predictions_lstm = [item for sublist in (model.predict(X_test_lstm) > 0.5).astype("int32") \
                                for item in sublist]

In [43]:
print("Neural networks - LSTM")
print_performance_report(y_test, predictions_lstm)

Neural networks - LSTM
	Accuracy: 0.86, Precision: 0.87, Recall: 0.85, F1 score: 0.86


### (7) Neural network-based approaches - Transformers

[Transformers](https://huggingface.co/transformers/quicktour.html) library offers a variety of implemented architectures (Tensorflow and PyTorch) along with [pre-trained models](https://huggingface.co/models) for different tasks - sequence classification, sequence tagging, machine translation, .... There you can find also some Slovene models.

[A nice introduction into BERT](https://huggingface.co/blog/bert-101) (for reading).

Loading a pre-trained model

In [44]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing input

In [45]:
pad_token=0
pad_token_segment_id=0
max_length=128

def convert_to_input(reviews):
  input_ids,attention_masks,token_type_ids=[],[],[]
  
  for x in tqdm(reviews,position=0, leave=True):
    inputs = bert_tokenizer.encode_plus(x,add_special_tokens=True, max_length=max_length)
    
    i, t = inputs["input_ids"], inputs["token_type_ids"]
    m = [1] * len(i)

    padding_length = max_length - len(i)

    i = i + ([pad_token] * padding_length)
    m = m + ([0] * padding_length)
    t = t + ([pad_token_segment_id] * padding_length)
    
    input_ids.append(i)
    attention_masks.append(m)
    token_type_ids.append(t)
  
  return [np.asarray(input_ids), 
            np.asarray(attention_masks), 
            np.asarray(token_type_ids)]

In [46]:
X_test_input=convert_to_input(X_test)
X_train_input=convert_to_input(X_train)

100%|██████████| 35000/35000 [02:11<00:00, 266.29it/s]


In [47]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((X_train_input[0],X_train_input[1],X_train_input[2],y_train)).map(example_to_features).shuffle(100).batch(12).repeat(5)
test_ds=tf.data.Dataset.from_tensor_slices((X_test_input[0],X_test_input[1],X_test_input[2],y_test)).map(example_to_features).batch(12)

Defining the model

In [48]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Training

In [49]:
bert_history = bert_model.fit(train_ds, epochs=2)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f4af5ebc750> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f4af5ebc750> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

14585/14585 [==============================] - 4644s 317ms/step - loss: 0.3493 - accuracy: 0.8388
Epoch 2/2
14585/14585 [==============================] - 4643s 318ms/step - loss: 0.2829 - accuracy: 0.8544


Predicting

In [50]:
results = bert_model.predict(test_ds)
predictions_bert = np.argmax(results.logits, axis=1)

In [51]:
print("Neural networks - BERT")
print_performance_report(y_test, predictions_bert)

Neural networks - BERT
	Accuracy: 0.51, Precision: 0.51, Recall: 1.0, F1 score: 0.67


## Conclusion

Based on the above, we got the following results:

| Model                             |Accuracy | Precision | Recall | F1 score |
|-----------------------------------|:-------:|:---------:|:------:|:--------:|
|Random                             |0.5      | 0.5       | 0.5    | 0.5      |
|Rule-based                         |0.66     | 0.83      | 0.41   | 0.55     |
|Neural networks - BERT (2 epochs)  |0.51     | 0.51      | 1.0    | 0.67     |
|Rule-based approach (lemmatization)|0.73     | 0.74      | 0.72   | 0.73     |
|naive Bayes (bag of words)         |0.82     | 0.82      | 0.82   | 0.82     |
|Logistic regression (bag of words) |0.86     | 0.86      | 0.87   | 0.86     |
|Neural networks - LSTM             |0.86     | 0.87      | 0.85   | 0.86     |
|Neural networks - BERT (5 epochs)  |0.88     | 0.89      | 0.87   | 0.88     |
|Logistic regression (TF-IDF)       |0.9      | 0.89      | 0.91   | 0.9      |



Take-away message: **Do not over-engineer or use heavy machinery unless needed!**

## Hands-on example

In [52]:
def movie_sentiment(reviews):
    # TODO: pretty print the results of all the algorithms above for given reviews
    pass
    
    
reviews = []

reviews.append('I like it')
reviews.append('I dont like it')
reviews.append('After 30 min I still did not know what the movie is about')
reviews.append('It is so good that I will never ever watch it again. Boring experience.')
reviews.append("It is like the Titanic movie!")
reviews.append("That is the best movie I have ever seen.")
reviews.append("That is the worst movie I have ever seen.")

print(movie_sentiment(reviews))

None
